### Checking dataset number of frames


In [ ]:
import os
import csv
import random
from decord import cpu, gpu
from decord import VideoReader
from tqdm import tqdm 

processed_dataset_path = "datasets/VI_dataset_mix_224_10s/"
# processed_dataset_path = "datasets/MIX_DATA/"
vid_paths = os.listdir(processed_dataset_path)
print(len(vid_paths))
frames = 0
for vid_path in tqdm(vid_paths):
    if vid_path[-1]=='t':
        print(vid_path)
        continue
    vid_path = os.path.join(processed_dataset_path, vid_path)
    # print(vid_path)
    vr = VideoReader(vid_path, ctx=cpu(0))
    frames += len(vr)
    
print("Total frames =", frames)
print("Total time =", frames/(30*3600))

# MIX_DATA - 
# Total long videos = 18 
# Total frames = 1603628
# Total time = 14.85 hour
# MIX_Dataset -
# Total short videos = 5269
# Each of 10 sec - 30 fps
# Total frames = 1580700
# 14.64 hours

### Dataset Split


In [ ]:
processed_dataset_path = "datasets/VI_dataset_mix_224_10s/"
split_file_path = os.path.join(processed_dataset_path, "splits.csv")
vid_paths = os.listdir(processed_dataset_path)
print(len(vid_paths))

In [ ]:
train_size = int(len(vid_paths)*0.8)
print(train_size, len(vid_paths)-train_size)
random.shuffle(vid_paths)
    
with open(split_file_path, 'w', newline='')as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(['name', 'split'])
    for i, v in enumerate(vid_paths):
        if i < train_size:
            csvwriter.writerow([v, 'train'])
        else:
            csvwriter.writerow([v, 'test'])

### Edge loss

In [ ]:
import cv2
import numpy as np
import torch
from matplotlib import pyplot as plt

def create_edge_map(image_tensor):
    # image_np = image_tensor.transpose(1, 2, 0) * 255  # Convert to numpy format
    image_gray = cv2.cvtColor(image_tensor.astype(np.uint8), cv2.COLOR_RGB2GRAY)

    # Apply Sobel filter to get edges
    edge_x = cv2.Sobel(image_gray, cv2.CV_64F, 1, 0, ksize=5)
    edge_y = cv2.Sobel(image_gray, cv2.CV_64F, 0, 1, ksize=5)
    edge_map = np.sqrt(edge_x**2 + edge_y**2)
    # edge_map = torch.tensor(edge_map / edge_map.max(), dtype=torch.float32)
    edge_map = edge_map / edge_map.max()
    
    return edge_map

In [ ]:
img = cv2.imread('./panda.png')
# print(img.shape)
plt.imshow(img)
plt.imshow(create_edge_map(img), cmap='gray')


In [ ]:
b = np.array([img, img])#.rehsape()
# b = np.concatenate([img,img], )
print(b.shape, img.shape)

image_gray = cv2.cvtColor(b.astype(np.uint8), cv2.COLOR_RGB2GRAY)


#### Using torch

In [ ]:
import torch.nn.functional as F
from torchvision.transforms import Grayscale, ToPILImage, ToTensor

pil = ToPILImage()
gs = Grayscale()
tensor = ToTensor()
def detect_edges(img):
    # Convert the image to grayscale (assumes img is a torch tensor with shape [C, H, W])
    # gray_img = img.mean(dim=0, keepdim=True)  # Convert to single-channel grayscale
    # gray_img = tensor(gs(pil(img)))
    gray_img = gs(img)
    print(gray_img.shape)
    # Apply Sobel filter for edge detection in x and y directions
    sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3)
    sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).view(1, 1, 3, 3)
    
    edge_x = F.conv2d(gray_img, sobel_x, padding=1)
    edge_y = F.conv2d(gray_img, sobel_y, padding=1)
    
    # Combine edges and normalize
    edge_magnitude = torch.sqrt(edge_x**2 + edge_y**2).squeeze(0)
    edge_mask = torch.clamp(edge_magnitude, 0, 1)  # Ensure values are in range [0, 1]
    
    return edge_mask

In [ ]:
t_img = torch.tensor(np.array([img, img])).permute(0, 3, 1, 2)
# t_img.shape
detect_edges(t_img)